# pLTV Python Model

## TO DO LIST

1. Add all forecasting methods to model. - DONE
2. Add back testing to model.
3. Revisit loan size forecast, errors seem large.

#### Resources

Github repo
https://github.com/kliao-tala/pLTV

Fader & Hardie paper on sBG model
https://drive.google.com/file/d/1tfMiERon1HgWo8dDJddwSzwc_AXK7LCA/view?usp=sharing
https://faculty.wharton.upenn.edu/wp-content/uploads/2012/04/Fader_hardie_jim_07.pdf

YT tutorial to pull look data
https://www.youtube.com/watch?v=EKwtLBnwXHk&list=PLXwS3L4W3KR2fhnQa-sLyajPZ9-L00KGX&index=1

Looker Python SDK & API
https://inventure.looker.com/extensions/marketplace_extension_api_explorer::api-explorer/4.0

---

In [14]:
import numpy as np

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

from scipy.optimize import curve_fit, minimize
from scipy.special import logsumexp

import plotly
from plotly import graph_objects as go
import plotly.io as pio

# change default plotly theme
pio.templates.default = "plotly_white"

import sbg

In [2]:
inputs = pd.read_csv('data/pltv_inputs.csv')
data = pd.read_csv('data/ke_data.csv')
pltv_expected = pd.read_csv('data/pltv_expected.csv')

## Data Munging

In [3]:
# fix date inconsistencies
data = data.replace({'2021-9': '2021-09', '2021-8': '2021-08', '2021-7': '2021-07', 
              '2021-6': '2021-06', '2021-5': '2021-05', '2021-4': '2021-04', '2020-9': '2020-09'})

# sort by months since first disbursement
data = data.sort_values(['First Loan Local Disbursement Month', 
                         'Months Since First Loan Disbursed'])

# remove all columns calculated through looker
data = data.loc[:,:"Default Rate Amount 51D"]

In [4]:
data.head()

,First Loan Local Disbursement Month,Months Since First Loan Disbursed,Count First Loans,Count Borrowers,Count Loans,Total Amount,Total Interest Assessed,Total Rollover Charged,Total Rollover Reversed,Default Rate Amount 7D,Default Rate Amount 30D,Default Rate Amount 51D
0,2020-09,0,7801,7801,13156,48361000,6540240,681325,81520,0.155382,0.121192,0.113031
15,2020-09,1,0,4481,5697,34490000,4660880,416544,32387,0.130661,0.101823,0.095738
2,2020-09,2,0,3661,4310,31461000,4297310,401077,30617,0.139719,0.103958,0.094792
9,2020-09,3,0,3050,3599,30482000,4178400,343062,17629,0.125111,0.089089,0.084399
10,2020-09,4,0,2549,2985,29303000,3964590,300262,920,0.113720,0.086750,0.081658


## pLTV Model

In [5]:
# KSH to USD conversion factor
ksh_usd = 0.00925
late_fee = 0.08 # % fee on defaults

In [6]:
# model parameters
min_months = 4

In [148]:
class Model:
    """
    sBG model class containing all functionality for creating, analyzing, and backtesting
    the sBG model.
    
    Parameters
    ----------
    data : pandas DataFrame
        Raw data pulled from Looker.
        
    Methods
    -------
    clean_data
        Performs all data cleaning steps and returns the cleaned data.
        
    borrower_retention(cohort_data)
        Computes borrower retention.
    
    """
    
    def __init__(self, data):
        self.data = data

        self.clean_data()
        
        
    def clean_data(self):
        # fix date inconsistencies
        self.data = self.data.replace({'2021-9': '2021-09', '2021-8': '2021-08', \
                                       '2021-7': '2021-07', '2021-6': '2021-06', \
                                       '2021-5': '2021-05', '2021-4': '2021-04', \
                                       '2020-9': '2020-09'})

        # sort by months since first disbursement
        self.data = self.data.sort_values(['First Loan Local Disbursement Month', 
                                 'Months Since First Loan Disbursed'])

        # remove all columns calculated through looker
        self.data = self.data.loc[:,:"Default Rate Amount 51D"]
        
        # add more convenient cohort column
        self.data['cohort'] = self.data['First Loan Local Disbursement Month']
        
        
    # --- DATA FUNCTIONS --- #
    def borrower_retention(self, cohort_data):
        return cohort_data['Count Borrowers']/cohort_data['Count Borrowers'].max()

    
    def borrower_survival(self, cohort_data):
        return cohort_data['Count Borrowers']/cohort_data['Count Borrowers'].shift(1)
    
    
    def loans_per_borrower(self, cohort_data):
        return cohort_data['Count Loans']/cohort_data['Count Borrowers']
    
    
    def loan_size(self, cohort_data, to_usd):
        df = cohort_data['Total Amount']/cohort_data['Count Loans']
        if to_usd:
            df *= ksh_usd
        return df
    
    
    def interest_rate(self, cohort_data):
        return cohort_data['Total Interest Assessed']/cohort_data['Total Amount']
    
    
    def default_rate(self, cohort_data, period=7):
        if period==7:
            return cohort_data['Default Rate Amount 7D'].fillna(0)
        
        elif period==51:
            default_rate = cohort_data['Default Rate Amount 51D']

            recovery_rate_51 = float(inputs[inputs.market=='ke']['recovery_7-30'] + \
                                     inputs[inputs.market=='ke']['recovery_30-51'])

            ## fill null 51dpd values with 7dpd values based on recovery rates
            derived_51dpd = (cohort_data['Count Loans']*(cohort_data['default_rate_7dpd']) - \
                cohort_data['Count Loans']*(cohort_data['default_rate_7dpd'])*recovery_rate_51)/ \
                cohort_data['Count Loans']
            
            return default_rate.fillna(derived_51dpd)
        
        elif period==365:
            # get actual data if it exists
            default_rate = np.nan*cohort_data['Default Rate Amount 51D']

            recovery_rate_365 = float(inputs[inputs.market=='ke']['recovery_51_'])

            ## fill null 365dpd values with 51dpd values based on recovery rates
            derived_365dpd = (cohort_data['Count Loans']*(cohort_data['default_rate_51dpd']) - \
                cohort_data['Count Loans']*(cohort_data['default_rate_51dpd'])* \
                recovery_rate_365)/cohort_data['Count Loans']

            return default_rate.fillna(derived_365dpd)
        
        
    def loans_per_original(self, cohort_data):
        return cohort_data['Count Loans']/cohort_data['Count Borrowers'].max()
    
    
    def origination_per_original(self, cohort_data, to_usd):
        df = cohort_data['Total Amount']/cohort_data['Count Borrowers'].max()
        if to_usd:
            df *= ksh_usd
        return df
    
    
    def revenue_per_original(self, cohort_data, to_usd):
        interest_revenue = cohort_data['origination_per_original']*cohort_data['interest_rate']
        
        # 0.08 is the % fee we charge to defaulted customers
        revenue = interest_revenue + (cohort_data['origination_per_original'] + interest_revenue) * \
            cohort_data['default_rate_7dpd']*0.08
        
        # note that origination_per_original is already in USD so no conversion is necessary
        return revenue
    
    
    def credit_margin(self, cohort_data):
        return cohort_data['revenue_per_original'] - \
                (cohort_data['origination_per_original'] + cohort_data['revenue_per_original'])* \
                cohort_data['default_rate_365dpd']
    
    
    def opex_per_original(self, cohort_data):
        opex_cost_per_loan = float(inputs[inputs.market=='ke']['opex cost per loan'])
        cost_of_capital = float(inputs[inputs.market=='ke']['cost of capital'])/12
        
        return opex_cost_per_loan*cohort_data['loans_per_original'] + \
            cost_of_capital*cohort_data['origination_per_original']
    
    
    def ltv_per_original(self, cohort_data):
        return cohort_data['cm$_per_original'] - cohort_data['opex_per_original']
    
    
    def credit_margin_percent(self, cohort_data):
        return cohort_data['ltv_per_original']/cohort_data['revenue_per_original']
        
        
    def generate_features(self, to_usd=True):
        """
        Generate all features required for pLTV model.
        """
        cohorts = []

        # for each cohort
        for cohort in self.data.loc[:,'First Loan Local Disbursement Month'].unique():
            # omit the last two months of incomplete data
            cohort_data = self.data[self.data['First Loan Local Disbursement Month']==cohort].iloc[:-2,:]

            # call data functions to generate calculated features
            cohort_data['borrower_retention'] = self.borrower_retention(cohort_data)
            cohort_data['borrower_survival'] = self.borrower_survival(cohort_data)
            cohort_data['loans_per_borrower'] = self.loans_per_borrower(cohort_data)
            cohort_data['loan_size'] = self.loan_size(cohort_data, to_usd)
            cohort_data['interest_rate'] = self.interest_rate(cohort_data)
            cohort_data['default_rate_7dpd'] = self.default_rate(cohort_data, period=7)
            cohort_data['default_rate_51dpd'] = self.default_rate(cohort_data, period=51)
            cohort_data['default_rate_365dpd'] = self.default_rate(cohort_data, period=365)
            cohort_data['loans_per_original'] = self.loans_per_original(cohort_data)
            cohort_data['origination_per_original'] = self.origination_per_original(cohort_data, to_usd)
            cohort_data['revenue_per_original'] = self.revenue_per_original(cohort_data, to_usd)
            cohort_data['cm$_per_original'] = self.credit_margin(cohort_data)
            cohort_data['opex_per_original'] = self.opex_per_original(cohort_data)
            cohort_data['ltv_per_original'] = self.ltv_per_original(cohort_data)
            cohort_data['cm%_per_original'] = self.credit_margin_percent(cohort_data)
            
            # reset the index and append the data
            cohorts.append(cohort_data.reset_index(drop=True))

        self.cohorts = cohorts
        self.data = pd.concat(cohorts, axis=0)
    
    
    def plot_cohorts(self, param, data='raw'):
        """
        Generate scatter plot for a specific paramter.
        
        Parameters
        ----------
        
        """
        
        curves = []
        if data == 'forecast' or data == 'backtest':
            if data == 'forecast':
                df = self.forecast
            elif data == 'backtest':
                df = self.backtest
                
            for cohort in df.cohort.unique():
                c_data = df[df.cohort==cohort]
                for dtype in c_data.data_type.unique():
                    output = c_data[c_data.data_type==dtype][param]

                    output.name = cohort + '-' + dtype

                    curves.append(output)
                
        elif data == 'raw':
            for cohort in self.data.cohort.unique():
                output = self.data[self.data.cohort==cohort][param]

                output.name = cohort

                curves.append(output)
            
        traces = []

        for cohort in curves:
            if 'forecast' in cohort.name:
                traces.append(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='lines',
                                        line=dict(width=3, dash='dash')))
            else:
                traces.append(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='markers+lines',
                                        line=dict(width=2)))

        fig = go.Figure(traces)
        fig.update_layout(xaxis=dict(title='Month Since Disbursement'),
                         yaxis=dict(title=param))

        fig.show()
        
        
    # --- FORECAST FUNCTIONS --- #
    def forecast_features(self, data, months=24, to_usd=True):
        """
        Generates a forecast of "Count Borrowers" out to the input number of months.
        The original and forecasted values are returned as a new dataframe, set as
        a new attribute of the model, *.forecast*. 
        
        Parameters
        ----------
        months : int
            Number of months to forecast to.
        """
        
        # initialize alpha and beta, optimized later by model
        alpha = beta = 1
        
        # list to hold individual cohort forecasts
        forecast_dfs = []

        # range of desired time periods
        times = list(range(1, months+1))
        times_dict = {i:i for i in times}
        
        for cohort in data.cohort.unique():
            # data for current cohort
            c_data = data[data.cohort==cohort]
            
            # starting cohort size
            n = c_data.loc[0, 'Count Borrowers']

            # only for cohorts with at least 4 data points
            if len(c_data) >= min_months:
                c = c_data['Count Borrowers']

                # define bounds for alpha and beta (must be positive)
                bounds = ((0,1e5), (0,1e5))
                
                # use scipy's minimize function on log_likelihood to optimize alpha and beta
                results = minimize(log_likelihood, np.array([alpha,beta]), args=c, bounds=bounds)

                
                # list to hold forecasted values 
                forecast = []
                for t in times:
                    forecast.append(n*s(t, results.x[0], results.x[1]))

                # convert list to dataframe
                forecast = pd.DataFrame(forecast, index=times, columns=['Count Borrowers'])

                # null df used to extend original cohort df to desired number of forecast months
                dummy_df = pd.DataFrame(np.nan, index=range(0,months+1), columns=['null'])

                # create label column to denote actual vs forecast data
                c_data['data_type'] = 'actual'

                # extend cohort df
                c_data = pd.concat([c_data, dummy_df], axis=1).drop('null', axis=1)
                
                # fill missing values in each col
                c_data.cohort = c_data.cohort.ffill()
                c_data['First Loan Local Disbursement Month'] = \
                    c_data['First Loan Local Disbursement Month'].ffill()
                c_data['Months Since First Loan Disbursed'] = \
                    c_data['Months Since First Loan Disbursed'].fillna(times_dict).astype(int)
                c_data['Count First Loans'] = c_data['Count First Loans'].ffill()
                

                # label forecasted data
                c_data.data_type = c_data.data_type.fillna('forecast')

                # fill in the forecasted data
                c_data['Count Borrowers'] = c_data['Count Borrowers'].fillna(forecast['Count Borrowers'])
                
                # add retention & survival features
                c_data['borrower_retention'] = m.borrower_retention(c_data)
                c_data['borrower_survival'] = m.borrower_survival(c_data)
                
                
                # forecast loan size
                for i in c_data[c_data.loan_size.isnull()].index:
                    c_data.loc[i, 'loan_size'] = c_data.loc[i-1, 'loan_size'] * \
                    pltv_expected.loc[i,'loan_size']/pltv_expected.loc[i-1, 'loan_size']
                
                
                # forecast loans_per_borrower
                for i in c_data[c_data.loans_per_borrower.isnull()].index:
                    c_data.loc[i, 'loans_per_borrower'] = c_data.loc[i-1, 'loans_per_borrower'] * \
                    pltv_expected.loc[i,'loans_per_borrower']/pltv_expected.loc[i-1, 'loans_per_borrower']
                
                
                # forecast Count Loans
                c_data['Count Loans'] = c_data['Count Loans'].fillna((c_data['loans_per_borrower'])*c_data['Count Borrowers'])
                
                
                # forecast Total Amount
                c_data['Total Amount'] = c_data['Total Amount'].fillna((c_data['loan_size']/ksh_usd)*c_data['Count Loans'])
                
                
                # forecast Interest Rate
                for i in c_data[c_data.interest_rate.isnull()].index:
                    c_data.loc[i, 'interest_rate'] = c_data.loc[i-1, 'interest_rate'] * \
                    pltv_expected.loc[i,'interest_rate']/pltv_expected.loc[i-1, 'interest_rate']
                
                
                # forecast default rate 7dpd
                default = c_data.default_rate_7dpd.dropna()
                default.index = np.arange(1, len(default)+1)
                
                def func(t, A, B):
                    return A*(t**B)

                params, covs = curve_fit(func, default.index, default)
                    
                t = list(range(1, months+2))
                fit = func(t, params[0], params[1])
                fit = pd.Series(fit, index=t).reset_index(drop=True)
                
                c_data['default_rate_7dpd'] = c_data['default_rate_7dpd'].fillna(fit)
                
                
                # derive 51dpd and 365 dpd from 7dpd
                c_data['default_rate_51dpd'] = m.default_rate(c_data, period=51)
                c_data['default_rate_365dpd'] = m.default_rate(c_data, period=365)
                
                # compute remaining columns from forecasts
                c_data['loans_per_original'] = m.loans_per_original(c_data)
                c_data['origination_per_original'] = m.origination_per_original(c_data, to_usd)
                c_data['revenue_per_original'] = m.revenue_per_original(c_data, to_usd)
                c_data['cm$_per_original'] = m.credit_margin(c_data)
                c_data['opex_per_original'] = m.opex_per_original(c_data)
                c_data['ltv_per_original'] = m.ltv_per_original(c_data)
                c_data['cm%_per_original'] = m.credit_margin_percent(c_data)
                
                
                
                # add the forecasted data for the cohort to a list, aggregating all cohort forecasts
                forecast_dfs.append(c_data)

        return pd.concat(forecast_dfs)
    
    
    def backtest(self, data, months=4, metric='mape'):
        """
        Backtest forecasted values against actuals.
        
        Parameters
        ----------
        
        
        """
        
        
        
        # --- Generate limited data --- #
        limited_data = []
        for cohort in data.cohort.unique():
            # data for current cohort
            c_data = data[data.cohort==cohort]

            # only backtest if remaining data has at least 4 data points
            if len(c_data) - months >= min_months:
                # limit data
                c_data = c_data.iloc[:len(c_data)-months,:]
                
                # forecast the limited data
                limited_data.append(m.forecast_features(c_data))
        
        backtest_data = pd.concat(limited_data)
        
        # add backtest prefix to col name
        for col in cols:
            backtest_data.rename(columns={col: f'bt-{col}'}, inplace=True)
        
        # --- Compute errors --- #
        def compute_error(actual, forecast, metric='mape'):
            """
            Test forecast performance against actuals using method defined by metric.
            """
            if metric=='rmse':
                error = np.sqrt((1/len(actual))*sum((forecast[:len(actual)] - actual)**2))
            elif metric=='mae':
                error = np.mean(abs(forecast[:len(actual)] - actual))
            elif metric=='mape':
                error = (1/len(actual))*sum(abs(forecast[:len(actual)] - actual)/actual)
            return error
        
        # append actuals to backtest dataframe
        for c in cols:
            
        
                
        return backtest_data
    

### Run Model

In [149]:
import warnings
warnings.simplefilter('error')

# create a model object
m = Model(data)

# generate features
m.generate_features()

# generate forecasts and save as model attribute
m.forecast = m.forecast_features(m.data)

# backtest 
m.backtest = m.backtest(m.data, months=6)

m.data.head()

,First Loan Local Disbursement Month,Months Since First Loan Disbursed,Count First Loans,Count Borrowers,Count Loans,Total Amount,Total Interest Assessed,Total Rollover Charged,Total Rollover Reversed,Default Rate Amount 7D,...,default_rate_7dpd,default_rate_51dpd,default_rate_365dpd,loans_per_original,origination_per_original,revenue_per_original,cm$_per_original,opex_per_original,ltv_per_original,cm%_per_original
0,2020-09,0,7801,7801,13156,48361000,6540240,681325,81520,0.155382,...,0.155382,0.113031,0.099467,1.686450,57.343834,8.564274,2.008581,2.847339,-0.838759,-0.097937
1,2020-09,1,0,4481,5697,34490000,4660880,416544,32387,0.130661,...,0.130661,0.095738,0.084249,0.730291,40.896359,6.011869,2.059888,1.407028,0.652860,0.108595
2,2020-09,2,0,3661,4310,31461000,4297310,401077,30617,0.139719,...,0.139719,0.094792,0.083417,0.552493,37.304737,5.569447,1.993030,1.133426,0.859604,0.154343
3,2020-09,3,0,3050,3599,30482000,4178400,343062,17629,0.125111,...,0.125111,0.084399,0.074271,0.461351,36.143892,5.365869,2.282891,1.000542,1.282349,0.238982
4,2020-09,4,0,2549,2985,29303000,3964590,300262,920,0.113720,...,0.113720,0.081658,0.071859,0.382643,34.745898,5.059868,2.199469,0.881503,1.317966,0.260474


In [186]:
m.data[cols].set_index('First Loan Local Disbursement Month', append=True)

KeyError: "None of ['cohort'] are in the columns"

In [181]:
m.backtest

,First Loan Local Disbursement Month,Months Since First Loan Disbursed,Count First Loans,bt-Count Borrowers,bt-Count Loans,bt-Total Amount,Total Interest Assessed,Total Rollover Charged,Total Rollover Reversed,Default Rate Amount 7D,...,bt-default_rate_51dpd,bt-default_rate_365dpd,bt-loans_per_original,bt-origination_per_original,bt-revenue_per_original,bt-cm$_per_original,bt-opex_per_original,bt-ltv_per_original,bt-cm%_per_original,data_type
0,2020-09,0,7801.0,7801.000000,13156.000000,4.836100e+07,6540240.0,681325.0,81520.0,0.155382,...,0.113031,0.099467,1.686450,57.343834,8.564274,2.008581,2.847339,-0.838759,-0.097937,actual
1,2020-09,1,0.0,4481.000000,5697.000000,3.449000e+07,4660880.0,416544.0,32387.0,0.130661,...,0.095738,0.084249,0.730291,40.896359,6.011869,2.059888,1.407028,0.652860,0.108595,actual
2,2020-09,2,0.0,3661.000000,4310.000000,3.146100e+07,4297310.0,401077.0,30617.0,0.139719,...,0.094792,0.083417,0.552493,37.304737,5.569447,1.993030,1.133426,0.859604,0.154343,actual
3,2020-09,3,0.0,3050.000000,3599.000000,3.048200e+07,4178400.0,343062.0,17629.0,0.125111,...,0.084399,0.074271,0.461351,36.143892,5.365869,2.282891,1.000542,1.282349,0.238982,actual
4,2020-09,4,0.0,2549.000000,2985.000000,2.930300e+07,3964590.0,300262.0,920.0,0.113720,...,0.081658,0.071859,0.382643,34.745898,5.059868,2.199469,0.881503,1.317966,0.260474,actual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,2021-04,20,0.0,1307.124721,1514.201661,3.658785e+07,NaN,NaN,NaN,NaN,...,0.087458,0.076963,0.159440,35.636266,5.009473,1.881250,0.596520,1.284730,0.256460,forecast
21,2021-04,21,0.0,1276.993947,1479.297519,3.574446e+07,NaN,NaN,NaN,NaN,...,0.086909,0.076480,0.155765,34.814808,4.891664,1.854909,0.582770,1.272139,0.260063,forecast
22,2021-04,22,0.0,1248.894946,1446.747026,3.495794e+07,NaN,NaN,NaN,NaN,...,0.086388,0.076021,0.152337,34.048742,4.781860,1.829903,0.569947,1.259956,0.263487,forecast
23,2021-04,23,0.0,1222.608377,1416.296094,3.422215e+07,NaN,NaN,NaN,NaN,...,0.085892,0.075585,0.149131,33.332089,4.679192,1.806116,0.557950,1.248166,0.266748,forecast


In [179]:
act.set_index('cohort', append=True)

,,First Loan Local Disbursement Month,Months Since First Loan Disbursed,Count First Loans,Count Borrowers,Count Loans,Total Amount,Total Interest Assessed,Total Rollover Charged,Total Rollover Reversed,Default Rate Amount 7D,...,default_rate_7dpd,default_rate_51dpd,default_rate_365dpd,loans_per_original,origination_per_original,revenue_per_original,cm$_per_original,opex_per_original,ltv_per_original,cm%_per_original
,cohort,,,,,,,,,,,,,,,,,,,,,
0,2020-10,2020-10,0,9831,9831,16433,58188000,7925530,893074,78262,0.167008,...,0.167008,0.121820,0.107202,1.671549,54.749161,8.288257,1.530528,2.799561,-1.269033,-0.153112
1,2020-10,2020-10,1,0,5296,6533,38258000,5218190,545877,50115,0.157486,...,0.157486,0.109831,0.096651,0.664531,35.996999,5.425182,1.421678,1.267148,0.154530,0.028484
2,2020-10,2020-10,2,0,4369,5227,37132000,5084890,483007,26677,0.143298,...,0.143298,0.098922,0.087051,0.531685,34.937545,5.239746,1.742261,1.080315,0.661946,0.126332
3,2020-10,2020-10,3,0,3573,4215,34670000,4728190,401907,3122,0.128913,...,0.128913,0.084171,0.074070,0.428746,32.621046,4.831062,2.056981,0.919339,1.137642,0.235485
4,2020-10,2020-10,4,0,2953,3347,32338000,4434570,377769,0,0.130323,...,0.130323,0.092634,0.081518,0.340454,30.426864,4.533220,1.683341,0.779023,0.904317,0.199487
5,2020-10,2020-10,5,0,2667,3149,35534000,4807250,349718,190,0.109985,...,0.109985,0.076774,0.067561,0.320313,33.433984,4.857124,2.270147,0.785015,1.485132,0.305764
6,2020-10,2020-10,6,0,2377,2729,35402000,4776020,334180,0,0.104983,...,0.104983,0.076507,0.067326,0.277591,33.309785,4.811262,2.244730,0.727277,1.517453,0.315396
7,2020-10,2020-10,7,0,2181,2516,36099000,4852540,318918,184,0.098333,...,0.098333,0.073145,0.064367,0.255925,33.965594,4.868874,2.369197,0.705782,1.663415,0.341643
8,2020-10,2020-10,8,0,1937,2213,35016000,4674830,271642,0,0.085779,...,0.085779,0.061672,0.054271,0.225104,32.946597,4.654828,2.614149,0.654059,1.960090,0.421087


In [158]:
act = m.data[m.data.cohort=='2020-10']
act

,First Loan Local Disbursement Month,Months Since First Loan Disbursed,Count First Loans,Count Borrowers,Count Loans,Total Amount,Total Interest Assessed,Total Rollover Charged,Total Rollover Reversed,Default Rate Amount 7D,...,default_rate_7dpd,default_rate_51dpd,default_rate_365dpd,loans_per_original,origination_per_original,revenue_per_original,cm$_per_original,opex_per_original,ltv_per_original,cm%_per_original
0,2020-10,0,9831,9831,16433,58188000,7925530,893074,78262,0.167008,...,0.167008,0.121820,0.107202,1.671549,54.749161,8.288257,1.530528,2.799561,-1.269033,-0.153112
1,2020-10,1,0,5296,6533,38258000,5218190,545877,50115,0.157486,...,0.157486,0.109831,0.096651,0.664531,35.996999,5.425182,1.421678,1.267148,0.154530,0.028484
2,2020-10,2,0,4369,5227,37132000,5084890,483007,26677,0.143298,...,0.143298,0.098922,0.087051,0.531685,34.937545,5.239746,1.742261,1.080315,0.661946,0.126332
3,2020-10,3,0,3573,4215,34670000,4728190,401907,3122,0.128913,...,0.128913,0.084171,0.074070,0.428746,32.621046,4.831062,2.056981,0.919339,1.137642,0.235485
4,2020-10,4,0,2953,3347,32338000,4434570,377769,0,0.130323,...,0.130323,0.092634,0.081518,0.340454,30.426864,4.533220,1.683341,0.779023,0.904317,0.199487
5,2020-10,5,0,2667,3149,35534000,4807250,349718,190,0.109985,...,0.109985,0.076774,0.067561,0.320313,33.433984,4.857124,2.270147,0.785015,1.485132,0.305764
6,2020-10,6,0,2377,2729,35402000,4776020,334180,0,0.104983,...,0.104983,0.076507,0.067326,0.277591,33.309785,4.811262,2.244730,0.727277,1.517453,0.315396
7,2020-10,7,0,2181,2516,36099000,4852540,318918,184,0.098333,...,0.098333,0.073145,0.064367,0.255925,33.965594,4.868874,2.369197,0.705782,1.663415,0.341643
8,2020-10,8,0,1937,2213,35016000,4674830,271642,0,0.085779,...,0.085779,0.061672,0.054271,0.225104,32.946597,4.654828,2.614149,0.654059,1.960090,0.421087
9,2020-10,9,0,1773,2043,34648000,4654950,365713,0,0.119216,...,0.119216,0.085111,0.074898,0.207812,32.600346,4.732537,1.936377,0.627482,1.308894,0.276573


In [157]:
bt = m.backtest[m.backtest.cohort=='2020-10']
bt

,First Loan Local Disbursement Month,Months Since First Loan Disbursed,Count First Loans,bt-Count Borrowers,bt-Count Loans,bt-Total Amount,Total Interest Assessed,Total Rollover Charged,Total Rollover Reversed,Default Rate Amount 7D,...,bt-default_rate_51dpd,bt-default_rate_365dpd,bt-loans_per_original,bt-origination_per_original,bt-revenue_per_original,bt-cm$_per_original,bt-opex_per_original,bt-ltv_per_original,bt-cm%_per_original,data_type
0,2020-10,0,9831.0,9831.000000,16433.000000,5.818800e+07,7925530.0,893074.0,78262.0,0.167008,...,0.121820,0.107202,1.671549,54.749161,8.288257,1.530528,2.799561,-1.269033,-0.153112,actual
1,2020-10,1,0.0,5296.000000,6533.000000,3.825800e+07,5218190.0,545877.0,50115.0,0.157486,...,0.109831,0.096651,0.664531,35.996999,5.425182,1.421678,1.267148,0.154530,0.028484,actual
2,2020-10,2,0.0,4369.000000,5227.000000,3.713200e+07,5084890.0,483007.0,26677.0,0.143298,...,0.098922,0.087051,0.531685,34.937545,5.239746,1.742261,1.080315,0.661946,0.126332,actual
3,2020-10,3,0.0,3573.000000,4215.000000,3.467000e+07,4728190.0,401907.0,3122.0,0.128913,...,0.084171,0.074070,0.428746,32.621046,4.831062,2.056981,0.919339,1.137642,0.235485,actual
4,2020-10,4,0.0,2953.000000,3347.000000,3.233800e+07,4434570.0,377769.0,0.0,0.130323,...,0.092634,0.081518,0.340454,30.426864,4.533220,1.683341,0.779023,0.904317,0.199487,actual
5,2020-10,5,0.0,2667.000000,3149.000000,3.553400e+07,4807250.0,349718.0,190.0,0.109985,...,0.076774,0.067561,0.320313,33.433984,4.857124,2.270147,0.785015,1.485132,0.305764,actual
6,2020-10,6,0.0,2377.000000,2729.000000,3.540200e+07,4776020.0,334180.0,0.0,0.104983,...,0.076507,0.067326,0.277591,33.309785,4.811262,2.244730,0.727277,1.517453,0.315396,actual
7,2020-10,7,0.0,2181.000000,2516.000000,3.609900e+07,4852540.0,318918.0,184.0,0.098333,...,0.073145,0.064367,0.255925,33.965594,4.868874,2.369197,0.705782,1.663415,0.341643,actual
8,2020-10,8,0.0,1937.000000,2213.000000,3.501600e+07,4674830.0,271642.0,0.0,0.085779,...,0.061672,0.054271,0.225104,32.946597,4.654828,2.614149,0.654059,1.960090,0.421087,actual
9,2020-10,9,0.0,1773.000000,2043.000000,3.464800e+07,4654950.0,365713.0,0.0,0.119216,...,0.085111,0.074898,0.207812,32.600346,4.732537,1.936377,0.627482,1.308894,0.276573,actual


In [162]:
idx_start = bt[bt.data_type=='forecast'].index.min()
idx_start

10

In [166]:
idx_stop = act.index.max()
idx_stop

15

In [171]:
fcast = bt.loc[idx_start:idx_stop,'bt-Count Borrowers']

In [172]:
actual = act.loc[idx_start: idx_stop,'Count Borrowers']

In [192]:
test(actual=actual, forecast=fcast)

0.14228584666456146

In [173]:
cols = ['Count Borrowers', 'borrower_retention', 'borrower_survival', 'loan_size', 
                'loans_per_borrower', 'Count Loans', 'Total Amount', 'interest_rate', 'default_rate_7dpd',
                'default_rate_51dpd', 'default_rate_365dpd', 'loans_per_original', 
                'origination_per_original', 'revenue_per_original', 'cm$_per_original',
                'opex_per_original', 'ltv_per_original', 'cm%_per_original']

In [177]:
m.data[m.data.columns.difference(cols)]

,Count First Loans,Default Rate Amount 30D,Default Rate Amount 51D,Default Rate Amount 7D,First Loan Local Disbursement Month,Months Since First Loan Disbursed,Total Interest Assessed,Total Rollover Charged,Total Rollover Reversed,cohort
0,7801,0.121192,0.113031,0.155382,2020-09,0,6540240,681325,81520,2020-09
1,0,0.101823,0.095738,0.130661,2020-09,1,4660880,416544,32387,2020-09
2,0,0.103958,0.094792,0.139719,2020-09,2,4297310,401077,30617,2020-09
3,0,0.089089,0.084399,0.125111,2020-09,3,4178400,343062,17629,2020-09
4,0,0.086750,0.081658,0.113720,2020-09,4,3964590,300262,920,2020-09
...,...,...,...,...,...,...,...,...,...,...
0,8843,0.184020,0.179847,0.200963,2021-11,0,6710270,858960,0,2021-11
1,0,0.165050,0.126838,0.185518,2021-11,1,4741570,441073,0,2021-11
2,0,0.176443,NaN,0.175424,2021-11,2,4221010,104591,0,2021-11
0,7818,0.182345,0.181757,0.210072,2021-12,0,6305570,698586,0,2021-12


In [188]:
# visualize cohorts for a given feature
m.plot_cohorts('default_rate_7dpd', data='actual')

## Forecasting

### Power law regression

In [ ]:
def power_law(x, A, B):
    return A*x**B

In [ ]:
arr = m.data[m.data['First Loan Local Disbursement Month']=='2020-12'].loc[1:, 'Count Loans']
arr = arr.dropna()
power_param, power_cov = curve_fit(power_law, arr.index, arr)

In [ ]:
x = list(range(1,25))
power_fit = power_law(x, power_param[0], power_param[1])

In [ ]:
traces = [
    go.Scatter(name='actual', x=arr.index, y=arr),
    go.Scatter(name='power-law', x=x, y=power_fit)
]

fig = go.Figure(traces)

fig.show()

### sBG probalistic model

sBG model assumptions:
1. The propensity of one customer to drop out is independent of the behavior of every other customer.
2. Individual customer retention rates are unchanged over time.
3. Observed retention increase with time due to aggregate results and heterogenity in customer behaviors.
4. Model applies for customer relationships in "discrete-time" and "contractual" settings.

In [ ]:
# initial guesses @ alpha and beta
alpha = 1
beta = 1

In [133]:
def p(t, alpha, beta):
    """
    Probability that a customer fails to take out another loan (probability to churn).
    For the derivation of this equation, see the original Fader & Hardie paper. This 
    recursion formula takes two constants, alpha and beta, which are fit to actual data.
    It then allows you to compute the probability of churn for a given time period, t.
    
    Parameters
    ----------
    t : int
        Time period.
    alpha : float
        Fitting parameter.
    beta : float
        Fitting parameter.
    
    Returns
    -------
    P : float
        Probability of churn.
    """
    
    eps = 1e-50
    
    if alpha + beta < eps:
        if t==1:
            return alpha/(eps)
        else:
            return p(t-1, alpha, beta) * (beta+t-2)/(eps+t-1)
    else:
        if t==1:
            return alpha/(alpha + beta)
        else:
            return p(t-1, alpha, beta) * (beta+t-2)/(alpha+beta+t-1)
    
def s(t, alpha, beta):
    """
    Survival function: the probability that a customer has survived to time t.
    For the derivation of this equation, see the original Fader & Hardie paper. This 
    recursion formula takes two constants, alpha and beta, which are fit to actual data.
    It also requires computation of P (probability of a customer churning).
    
    Parameters
    ----------
    t : int
        Time period.
    alpha : float
        Fitting parameter.
    beta : float
        Fitting parameter.
    
    Returns
    -------
    S : float
        Probability of survival.
    """
    
    if t==1:
        return 1 - p(t, alpha, beta)
    else:
        return s(t-1, alpha, beta) - p(t, alpha, beta)
    
def log_likelihood(params, c):
    """
    Computes the *negative* log-likelihood of the probability distribution of customers
    still being active at time t. For a derivation of the log-likelihood, see Appendix A
    in the original Fader & Hardie paper. The function computes the log-likelihood at 
    every time step, t, leading up to the last time period T. The final value is simply
    the sum of the log-likelihood computed at each time step. In the end, we return the 
    negative of the log-likelihood so that we can use scipy's minimize function to optimize
    for the values of alpha and beta.
    
    Parameters
    ----------
    params : array
        Array containing alpha and beta values.
    c : array
        Array containing borrower count for a given cohort.
    
    Returns
    -------
    ll : float
        log-likelihood value
    """
        
    alpha, beta = params
    eps = 1e-50
    
    # initialize log-likelihood (ll) value at 0
    ll=0
    
    # for each time period in the *actual* data, compute ll and add it to the running total
    for t in c[1:].index:
        if p(t, alpha, beta) < eps:
            ll += (c[t-1]-c[t])*np.log(eps)
        else:
            ll += (c[t-1]-c[t])*np.log(p(t, alpha, beta))
    
    # add the final term which associated with customers who are still active at the end
    # of the final period.
    if s((len(c)-1)-1, alpha, beta)-p(len(c)-1, alpha, beta) < eps:
        ll += c.iloc[-1]*np.log(eps)
    else:
        ll += c.iloc[-1]*np.log(s((len(c)-1)-1, alpha, beta)-p(len(c)-1, alpha, beta))
    
    return -ll

In [ ]:
cohort = '2020-09'

In [ ]:
c = m.backtest[m.backtest['First Loan Local Disbursement Month']==cohort]['Count Borrowers']
c = c.reset_index(drop=True)

In [ ]:
c

In [ ]:
# since we're working with logs, we need bounds for alpha and beta > 0.
bounds = ((0,None), (0,None))

results = minimize(log_likelihood, np.array([1,1]), args=c, bounds=bounds)
results

In [ ]:
alpha_opt, beta_opt = results.x

sBG_forecast = []
for i in x:
    sBG_forecast.append(s(i, alpha_opt, beta_opt))

In [ ]:
arr = m.data[m.data['First Loan Local Disbursement Month']==cohort].loc[1:, 'borrower_retention']
arr = arr.dropna()

power_param, power_cov = curve_fit(power_law, arr.index, arr)
power_fit = power_law(x, power_param[0], power_param[1])

In [ ]:
traces = [
    go.Scatter(name='actual', x=arr.index, y=arr),
    go.Scatter(name='power-law', x=x, y=power_fit),
    go.Scatter(name='sBG', x=x, y=sBG_forecast)
]

fig = go.Figure(traces)
fig.update_layout(xaxis=dict(title='Months Since First Loan'),
                 yaxis=dict(title='Retention (%)'))

fig.show()

In [ ]:
power_res = abs(power_fit[:len(arr)] - arr)
sbg_res = abs(sBG_forecast[:len(arr)] - arr)

traces = [
    go.Scatter(name='power-law', x=arr.index, y=power_res),
    go.Scatter(name='sBG', x=arr.index, y=sbg_res)
]

fig = go.Figure(traces)
fig.update_layout(title='Model Residuals', xaxis=dict(title='Months Since First Loan'))

fig.show()

#### Scale forecast

In [ ]:
forecast_dfs = []

alpha = beta = 1

x = list(range(1, 25))
df = m.data[['cohort', 'Count Borrowers']]
for cohort in df.cohort.unique():
    c_data = df[df.cohort==cohort]
    n = c_data.loc[0, 'Count Borrowers']
    
    # only for cohorts with at least 4 data points
    if len(c_data) > 3:
        c = c_data['Count Borrowers']

        # fit model
        bounds = ((0,None), (0,None))
        results = minimize(log_likelihood, np.array([alpha,beta]), args=c, bounds=bounds)

        # forecast
        forecast = []
        for i in x:
            forecast.append(n*s(i, results.x[0], results.x[1]))

        forecast = pd.DataFrame(forecast, index=x, columns=['Count Borrowers'])
    
        holder_df = pd.DataFrame(np.nan, index=range(0,25), columns=['null'])
        
        c_data['data_type'] = 'actual'
        
        c_data = pd.concat([c_data, holder_df], axis=1).drop('null', axis=1)
        c_data.cohort = c_data.cohort.ffill()

        c_data.data_type = c_data.data_type.fillna('forecast')
        
        c_data = c_data.fillna(forecast)
        
        c_data['borrower_retention'] = m.borrower_retention(c_data)
        
        forecast_dfs.append(c_data)
        
forecast = pd.concat(forecast_dfs)

In [ ]:
cohort='2021-06'

df = forecast[forecast.cohort==cohort]

traces = []
for dtype in df.data_type.unique():
    traces.append(go.Scatter(name=dtype, x=df[df.data_type==dtype].index, 
                             y=df[df.data_type==dtype]['borrower_retention'], mode='markers+lines'))
    
fig = go.Figure(traces)
fig.update_layout(xaxis=dict(title='Months Since Disbursement'),
                 yaxis=dict(title='Count Borrowers'))

fig.show()

### Backtest Framework

To backtest, we need to withold a range of months of data and forecast. Then compare the forecasted values to actuals.

1. Withold range of data.
2. Forecast.
3. Compute error of forecast to actuals of withheld data.

In [ ]:
m.forecast_features(m.data[m.data.cohort=='2020-09'])

In [191]:
def test(actual, forecast, metric='mape'):
    """
    Test forecast performance against actuals using method defined by metric.
    """
    if metric=='rmse':
        error = np.sqrt((1/len(actual))*sum((forecast[:len(actual)] - actual)**2))
    elif metric=='mae':
        error = np.mean(abs(forecast[:len(actual)] - actual))
    elif metric=='mape':
        error = (1/len(actual))*sum(abs(forecast[:len(actual)] - actual)/actual)
    return error


In [ ]:
test(arr, power_fit) 

In [ ]:
test(arr, sBG_forecast)

### Test all cohorts

In [ ]:
test_vals = {}

for cohort in m.data.cohort.unique():
    x = list(range(1,25))

    # power-law
    arr = m.data[m.data.cohort==cohort]['borrower_retention'][1:].dropna()
    
    if len(arr)>=3:
        power_param, power_cov = curve_fit(power_law, arr.index, arr)
        power_fit = power_law(x, power_param[0], power_param[1])

        # sbg
        c = m.data[m.data.cohort==cohort]['Count Borrowers'].reset_index(drop=True)

        # since we're working with logs, we need bounds for alpha and beta > 0.
        bounds = ((0,None), (0,None))

        results = minimize(log_likelihood, np.array([1,1]), bounds=bounds)

        sBG_forecast = []
        for i in x:
            sBG_forecast.append(s(i, results.x[0], results.x[1]))

        test_vals[cohort] = {'power-law': test(arr, power_fit, metric='rmse'), 
                             'sbg': test(arr, sBG_forecast)}
    
test_vals

In [ ]:
m.data[m.data.cohort=='2021-10']

In [ ]:
forecast_methods = {
    'Count Borrowers': 'sbg',
    'loans_per_borrwer': 'power-law'
}

In [ ]:
m.data[m.data['First Loan Local Disbursement Month']=='2020-09']['Count Borrowers']

#### Questions/Concerns

1. Is there a framework that's been developed/used to back test this forecast model?
2. Filter out bad cohorts. Why are some starting at 0? (e.g. 2021-07)
3. Why are the last few months not included? Why not just omit the final incomplete month?


In [ ]:
new_data[new_data['First Loan Local Disbursement Month'] == '2021-07']